In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Asgn4-dataset.csv")
print(df)

            ID                 Lesson  Student                    KC  \
0            0                  Splot  AGUFADE  VALUING-CAT-FEATURES   
1            1                  Splot  AGUFADE  VALUING-NUM-FEATURES   
2            2                  Splot  AGUFADE       CHOOSE-VAR-TYPE   
3            3                  Splot  AGUFADE  VALUING-NUM-FEATURES   
4            4                  Splot  AGUFADE       CHOOSE-VAR-TYPE   
...        ...                    ...      ...                   ...   
124365  128869  MidZSchoolZProbabilit  SKRDBGE           GREATEST-CF   
124366  128870  MidZSchoolZProbabilit  SKRDBGE           GREATEST-CF   
124367  128871  MidZSchoolZProbabilit  SKRDBGE           GREATEST-CF   
124368  128872  MidZSchoolZProbabilit  SKRDBGE           GREATEST-CF   
124369  128873  MidZSchoolZProbabilit  SKRDBGE             ENTER-GCD   

                               item  right  firstattempt   time  
0       META-VALUING-CAT-FEATURES-1      1             1  3.297  
1  

Question 1 of 10: Filter out all actions from (a copy of) the data set, until you only have actions for KC "VALUING-CAT-FEATURES". How many rows of data remain?

In [3]:
df_q1 = df[df["KC"] == "VALUING-CAT-FEATURES"]
print(len(df_q1))

2473


Question 3 of 10: Go ahead and delete the rows you indicated in question 2. How many rows of data remain?
(need to delete rows where first attempt is False -> because of BKT's assumptions)

In [4]:
df_q3 = df_q1[df_q1["firstattempt"] == 1]
print(len(df_q3))

1791


In [5]:
df_java = df_q3.reset_index(drop = True)
df_java = df_java.drop(columns=["ID", "firstattempt", "time", "item"])
df_java = df_java.rename(columns={
    "Lesson": "lesson",
    "Student": "student",
    "KC": "skill",
})
df_java["cell"] = "cell"
df_java["eol"] = "eol"
df_java = df_java.loc[:, ["lesson", "student", "skill", "cell", "right", "eol"]]
df_java.to_csv("java_exercise.txt", sep="\t", index_label="num")
df_java.to_csv("java_exercise_test.csv", index_label="num")
print(df_java)

     lesson  student                 skill  cell  right  eol
0     Splot  AGUFADE  VALUING-CAT-FEATURES  cell      1  eol
1     Splot  AGUFADE  VALUING-CAT-FEATURES  cell      0  eol
2     Splot  AOADMAC  VALUING-CAT-FEATURES  cell      1  eol
3     Splot  AOADMAC  VALUING-CAT-FEATURES  cell      0  eol
4     Splot  AOADMAC  VALUING-CAT-FEATURES  cell      0  eol
...     ...      ...                   ...   ...    ...  ...
1786  Splot  KBRIZPT  VALUING-CAT-FEATURES  cell      1  eol
1787  Splot  KBRIZPT  VALUING-CAT-FEATURES  cell      1  eol
1788  Splot  KBRIZPT  VALUING-CAT-FEATURES  cell      1  eol
1789  Splot  KBRIZPT  VALUING-CAT-FEATURES  cell      1  eol
1790  Splot  KBRIZPT  VALUING-CAT-FEATURES  cell      1  eol

[1791 rows x 6 columns]



Question 4 of 10: We're going to create a Bayesian Knowledge Tracing model for VALUING-CAT-FEATURES.

Create variable columns P(Ln-1) (cell I1), P(Ln-1|RESULT) (cell J1), and P(Ln) (cell K1), and leave the columns below them empty for now. (If you're not sure what these represent, re-watch the lecture.)

To the right of this, type into four cells, (cell M2) L0, (M3) T, (M4) S, and (M5) G.

Now type 0.3, 0.1, 0.2, and 0.25 to the right of (respectively) L0, T, S, and G (e.g. cells N2, N3, N4, N5). 

What is your slip parameter?

In [6]:
#df_q4 = df_q3.reindex(columns = df_q3.columns.tolist() + ["P(Ln-1)", "P(Ln-1) | RESULT", "P(Ln)"])
#P(Ln-1) - I1 - the probability that the student knows the skill before attempting the Q
#P(Ln-1) | RESULT - J1 - the probability that the student knew it before, given an answer
#P(Ln) - K1 - the probability that the student knows the skill after answering the question
L0 = 0.3 #the probability that the students knows the skill initially (skill = KC)
T = 0.1 #P that you will learn it if you did not know it 
S = 0.2 #slip
G = 0.25 #guess
print(f"slip is {S}")

slip is 0.2


My work

In [7]:
prior_name = "P(Ln-1)" 
conditioned_name = "P(Ln-1) | RESULT"
posterior_name = "P(Ln)"
df_my = df_q3.reindex(columns = df_q3.columns.tolist() + [prior_name, conditioned_name, posterior_name])
df_my["right"] = df_my["right"] == 1
#df_my = df_my.shift(periods=1)
df_my = pd.concat([pd.DataFrame({}, index=[-1]), df_my]).reset_index(drop=True) #add empty row on top
def bkt(df: pd.DataFrame):
    prior_p, conditioned_p, posterior_p = 0, 0, 0 #probabilities Pln-1, Pln-1 | R, Pln
    for i in range(1, len(df)):
        #finding the prior probability
        if df.loc[i-1, "Student"] == df.loc[i, "Student"]:
            prior_p = df.loc[i-1, posterior_name]
        else:
            prior_p = L0

        #finding the conditioned probability
        if df.loc[i, "right"]:
            p_did_not_slip_and_knew = (1 - S) * prior_p
            p_guessed_and_did_not_know = G * (1 - prior_p)
            conditioned_p = p_did_not_slip_and_knew / (p_did_not_slip_and_knew + p_guessed_and_did_not_know)
        else:
            p_slipped_and_knew = S * prior_p
            p_did_not_guess_and_did_not_know = (1-G) * (1 - prior_p)
            conditioned_p = p_slipped_and_knew / (p_slipped_and_knew + p_did_not_guess_and_did_not_know)

        

        df.loc[i, prior_name] = prior_p
        df.loc[i, conditioned_name] = conditioned_p

        #finding the posterior probability
        df.loc[i, posterior_name] = conditioned_p + T * (1 - conditioned_p)

In [8]:
bkt(df_my)
print(df_my)

           ID Lesson  Student                    KC  \
0         NaN    NaN      NaN                   NaN   
1         0.0  Splot  AGUFADE  VALUING-CAT-FEATURES   
2         7.0  Splot  AGUFADE  VALUING-CAT-FEATURES   
3       250.0  Splot  AOADMAC  VALUING-CAT-FEATURES   
4       253.0  Splot  AOADMAC  VALUING-CAT-FEATURES   
...       ...    ...      ...                   ...   
1787  71275.0  Splot  KBRIZPT  VALUING-CAT-FEATURES   
1788  71333.0  Splot  KBRIZPT  VALUING-CAT-FEATURES   
1789  71337.0  Splot  KBRIZPT  VALUING-CAT-FEATURES   
1790  71415.0  Splot  KBRIZPT  VALUING-CAT-FEATURES   
1791  71419.0  Splot  KBRIZPT  VALUING-CAT-FEATURES   

                             item  right  firstattempt    time   P(Ln-1)  \
0                             NaN    NaN           NaN     NaN       NaN   
1     META-VALUING-CAT-FEATURES-1   True           1.0   3.297  0.300000   
2     META-VALUING-CAT-FEATURES-2  False           1.0   3.203  0.620482   
3     META-VALUING-CAT-FEATURES-1  